In [1]:
__author__ = "Angus Liao 2023/2/24"

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
import os 

if not os.path.isdir('result_figures'): # 如果沒有這個路徑，建立一個資料夾
    os.mkdir('result_figures')

In [4]:
#用於印在圖表上的時段刻度
list_hour = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]

In [5]:
#各月份自然天數
N_m = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [6]:
#計算各月份資料起始的索引直
month_begin_index = []

index_counter = 0

for month in range(0, 12):
    if month == 0:
        month_begin_index.append(0)
    else:
        month_begin_index.append(index_counter+N_m[month-1]*24) #索引值=前一個月份起始索引值+前一個月份天數*24
        index_counter  = index_counter+N_m[month-1]*24

In [7]:
#讀入要進行視覺化的資料
Result = pd.read_excel("result/Result.xlsx")

#columns = Result.columns[3:] => 取第3行之後
Result = pd.DataFrame(Result, columns = Result.columns[3:])
    
Result = Result.drop(columns = [
    'Charge[month][date][hour]', 
    'Discharge[month][date][hour]', 
    'Purchase[month][date][hour]', 
    'Inventory[month][date][hour]', 
    'Inventory', 
    'PVoutput[month][date][hour]', 
    'Theta[month][date][hour]', 
    'Theta', 
    'Load[month][date][hour]', 
    'Price[month][date][hour]'
])

Result["Price"] = round(Result["Price"] / 30, 3)

Result

,Charge,Discharge,Purchase,PVoutput,Load,Price
0,0.000000,10908.34794,0.000000,0.0,1542,0.050
1,0.000000,1504.00000,0.000000,0.0,1504,0.050
2,0.000000,1487.00000,0.000000,0.0,1487,0.050
3,0.000000,1510.00000,0.000000,0.0,1510,0.050
4,0.000000,1506.00000,0.000000,0.0,1506,0.050
...,...,...,...,...,...,...
8779,624.888889,634.80000,3164.088889,0.0,3174,0.115
8780,522.222222,562.40000,2771.822222,0.0,2812,0.115
8781,413.111111,470.00000,2293.111111,0.0,2350,0.115
8782,355.333333,371.80000,1842.533333,0.0,1859,0.050


In [8]:
#春天3,4,5月

Spring_Charge, Spring_Discharge, Spring_Purchase, Spring_PVoutput, Spring_Load, Spring_Price = [], [], [], [], [], []

for hour in range(0, 24):  #24小時
    for month in range(2, 5):  #3,4,5月
        #一個小時一小時依序儲存，索引值一次跳24
        for index in range(month_begin_index[month]+hour, (month_begin_index[month]+(N_m[month]*24)), 24):
            #print(hour, month, index)  <--確認索引值用，不用理會
            
            #利用iloc依靠索引值來取得列資料，指定column name取得行資料
            Spring_Charge.append(Result.iloc[index, :]['Charge'])
            Spring_Discharge.append(Result.iloc[index, :]['Discharge'])
            Spring_Purchase.append(Result.iloc[index, :]['Purchase'])
            Spring_PVoutput.append(Result.iloc[index, :]['PVoutput'])
            Spring_Load.append(Result.iloc[index, :]['Load'])
            Spring_Price.append(Result.iloc[index, :]['Price'])

#將陣列轉換成numpy陣列，方便後續reshape用
Spring_hour_Charge = np.array(Spring_Charge)
Spring_hour_Discharge = np.array(Spring_Discharge)
Spring_hour_Purchase = np.array(Spring_Purchase)
Spring_hour_PVoutput = np.array(Spring_PVoutput)
Spring_hour_Load = np.array(Spring_Load)
Spring_hour_Price = np.array(Spring_Price)

#reshape成24*92的二維陣列 --> 24個小時，92天
Spring_hour_Charge = Spring_hour_Charge.reshape(24, 92)
Spring_hour_Discharge = Spring_hour_Discharge.reshape(24, 92)
Spring_hour_Purchase = Spring_hour_Purchase.reshape(24, 92)
Spring_hour_PVoutput = Spring_hour_PVoutput.reshape(24, 92)
Spring_hour_Load = Spring_hour_Load.reshape(24, 92)
Spring_hour_Price = Spring_hour_Price.reshape(24, 92)

#儲存平均值的陣列
Spring_average_Charge, Spring_average_Discharge, Spring_average_Purchase, Spring_average_PVoutput, Spring_average_Load, Spring_average_Price = [], [], [], [], [], []

#利用numpy average計算各項目各個小時的平均值
for hour in range(0, 24):
    Spring_average_Charge.append(round(np.average(Spring_hour_Charge[hour]), 4))
    Spring_average_Discharge.append(round(np.average(Spring_hour_Discharge[hour]), 4))
    Spring_average_Purchase.append(round(np.average(Spring_hour_Purchase[hour]), 4))
    Spring_average_PVoutput.append(round(np.average(Spring_hour_PVoutput[hour]), 4))
    Spring_average_Load.append(round(np.average(Spring_hour_Load[hour]), 4))
    Spring_average_Price.append(round(np.average(Spring_hour_Price[hour]), 4))
    

#製作季節性圖表
width1 = 0.25  #條狀圖單一條的寬度
plt.figure(figsize = (10, 10))  #設定圖表長寬
plt.subplot(2,1,1)  #每圖表以兩張圖表組合在一起，此為第一張
list1 = [x - width1/2 for x in range(len(list_hour))]  #因兩項資料放在一起，要共用資料寬度，所以一人一半
list2 = [x + width1/2 for x in range(len(list_hour))]
#bar(橫軸值, 縱軸值, 條寬度, 條顏色,資料標籤)
plt.bar(list1, Spring_average_Purchase, width1, color = "cyan", label = "Purchase")  #購電量資料
plt.bar(list2, Spring_average_Load, width1, color = "red", label = "Load")  #負載量資料
plt.xticks(range(len(list_hour)), labels = list_hour)  #x軸座標值
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)
plt.title("Spring Average")  #圖表標題
plt.xlabel("Hours")  #x軸座標標籤
plt.ylabel("kW/h")  #y軸座標標籤
plt.legend(loc = "upper left")  #資料標籤顯示位置
plt.grid()  #圖片結果以網格顯示
ax2 = plt.twinx()  #定義第二組不同呈現方式的資料
ax2.set_ylabel("US $/kWh")  #第二組y軸座標標籤
#plt(橫軸值, 縱軸值, 折線顏色與型態, 資料標籤)
plt.plot(list_hour, Spring_average_Price, "g-.*", label = "Price")  #購電價資料
plt.ylim(0, 0.2)
plt.legend(loc = "upper right")

width2 = 0.25
plt.subplot(2,1,2)
list1 = [x - width2/2 for x in range(len(list_hour))]  #因兩項資料放在一起，要共用資料寬度，所以一人一半
list2 = [x + width2/2 for x in range(len(list_hour))]
plt.bar(list1, Spring_average_Charge, width2, color = "orange", label = "Charge")  #充電量資料
plt.bar(list2, Spring_average_Discharge, width2, color = "blue", label = "Discharge")  #放電量資料
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)
plt.xlabel("Hours")
plt.ylabel("kW/h")
plt.legend(loc = "upper left")
plt.grid()
ax2 = plt.twinx()
ax2.set_ylabel("kW/h")
plt.plot(list_hour, Spring_average_PVoutput, "r-.*", label = "PV output")  #光伏輸出量資料
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)
plt.legend(loc = "upper right")

plt.savefig("result_figures/Spring.jpg")
plt.close()  #將製作完的圖表關閉，避免佔據記憶體空間

In [ ]:
#夏天6,7,8月
Summer_Charge, Summer_Discharge, Summer_Purchase, Summer_PVoutput, Summer_Load, Summer_Price = [], [], [], [], [], []

for hour in range(0, 24):
    for month in range(5, 8):
        for index in range(month_begin_index[month]+hour, (month_begin_index[month]+(N_m[month]*24)), 24):
            #print(hour, month, index)
            Summer_Charge.append(Result.iloc[index, :]['Charge'])
            Summer_Discharge.append(Result.iloc[index, :]['Discharge'])
            Summer_Purchase.append(Result.iloc[index, :]['Purchase'])
            Summer_PVoutput.append(Result.iloc[index, :]['PVoutput'])
            Summer_Load.append(Result.iloc[index, :]['Load'])
            Summer_Price.append(Result.iloc[index, :]['Price'])
            
Summer_hour_Charge = np.array(Summer_Charge)
Summer_hour_Discharge = np.array(Summer_Discharge)
Summer_hour_Purchase = np.array(Summer_Purchase)
Summer_hour_PVoutput = np.array(Summer_PVoutput)
Summer_hour_Load = np.array(Summer_Load)
Summer_hour_Price = np.array(Summer_Price)

Summer_hour_Charge = Summer_hour_Charge.reshape(24, 92)
Summer_hour_Discharge = Summer_hour_Discharge.reshape(24, 92)
Summer_hour_Purchase = Summer_hour_Purchase.reshape(24, 92)
Summer_hour_PVoutput = Summer_hour_PVoutput.reshape(24, 92)
Summer_hour_Load = Summer_hour_Load.reshape(24, 92)
Summer_hour_Price = Summer_hour_Price.reshape(24, 92)

Summer_average_Charge, Summer_average_Discharge, Summer_average_Purchase, Summer_average_PVoutput, Summer_average_Load, Summer_average_Price = [], [], [], [], [], []

for hour in range(0, 24):
    Summer_average_Charge.append(round(np.average(Summer_hour_Charge[hour]), 4))
    Summer_average_Discharge.append(round(np.average(Summer_hour_Discharge[hour]), 4))
    Summer_average_Purchase.append(round(np.average(Summer_hour_Purchase[hour]), 4))
    Summer_average_PVoutput.append(round(np.average(Summer_hour_PVoutput[hour]), 4))
    Summer_average_Load.append(round(np.average(Summer_hour_Load[hour]), 4))
    Summer_average_Price.append(round(np.average(Summer_hour_Price[hour]), 4))
    

width1 = 0.25  #條狀圖單一條的寬度
plt.figure(figsize = (10, 10))  #設定圖表長寬
plt.subplot(2,1,1)  #每日圖表以兩張圖表組合在一起，此為第一張
list1 = [x - width1/2 for x in range(len(list_hour))]  #因兩項資料放在一起，要共用資料寬度，所以一人一半
list2 = [x + width1/2 for x in range(len(list_hour))]
#.iloc[index_count:index_count+24] => 取得列索引值的對應資料
#["Purchase"] => 取得對應column name值的資料
#bar(橫軸值, 縱軸值, 條寬度, 條顏色,資料標籤)
plt.bar(list1, Summer_average_Purchase, width1, color = "cyan", label = "Purchase")  #購電量資料
plt.bar(list2, Summer_average_Load, width1, color = "red", label = "Load")  #負載量資料
plt.xticks(range(len(list_hour)), labels = list_hour)  #x軸座標值
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2000', '4000', '6000', '8000', '10,000']
plt.yticks(y, y_tick_labels)
plt.title("Summer Average")  #圖表標題
plt.xlabel("Hours")  #x軸座標標籤
plt.ylabel("kW/h")  #y軸座標標籤
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.legend(loc = "upper left")  #資料標籤顯示位置
plt.grid()  #圖片結果以網格顯示
ax2 = plt.twinx()  #定義第二組不同呈現方式的資料
ax2.set_ylabel("US $/kWh")  #第二組y軸座標標籤
#plt(橫軸值, 縱軸值, 折線顏色與型態, 資料標籤)
plt.plot(list_hour, Summer_average_Price, "g-.*", label = "Price")  #購電價資料
plt.ylim(0, 0.2)
plt.legend(loc = "upper right")

width2 = 0.25
plt.subplot(2,1,2)
list1 = [x - width2/2 for x in range(len(list_hour))]  #因兩項資料放在一起，要共用資料寬度，所以一人一半
list2 = [x + width2/2 for x in range(len(list_hour))]
plt.bar(list1, Summer_average_Charge, width2, color = "orange", label = "Charge")  #充電量資料
plt.bar(list2, Summer_average_Discharge, width2, color = "blue", label = "Discharge")  #放電量資料
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.xlabel("Hours")
plt.ylabel("kW/h")
plt.legend(loc = "upper left")
plt.grid()
ax2 = plt.twinx()
ax2.set_ylabel("kW/h")
plt.plot(list_hour, Summer_average_PVoutput, "r-.*", label = "PV output")  #光伏輸出量資料
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.legend(loc = "upper right")

plt.savefig("result_figures/Summer.jpg")
plt.close()  #將製作完的圖表關閉，避免佔據記憶體空間

In [ ]:
#秋天9,10,11
Autumn_Charge, Autumn_Discharge, Autumn_Purchase, Autumn_PVoutput, Autumn_Load, Autumn_Price = [], [], [], [], [], []

for hour in range(0, 24):
    for month in range(8, 11):
        for index in range(month_begin_index[month]+hour, (month_begin_index[month]+(N_m[month]*24)), 24):
            #print(hour, month, index)
            Autumn_Charge.append(Result.iloc[index, :]['Charge'])
            Autumn_Discharge.append(Result.iloc[index, :]['Discharge'])
            Autumn_Purchase.append(Result.iloc[index, :]['Purchase'])
            Autumn_PVoutput.append(Result.iloc[index, :]['PVoutput'])
            Autumn_Load.append(Result.iloc[index, :]['Load'])
            Autumn_Price.append(Result.iloc[index, :]['Price'])

Autumn_hour_Charge = np.array(Autumn_Charge)
Autumn_hour_Discharge = np.array(Autumn_Discharge)
Autumn_hour_Purchase = np.array(Autumn_Purchase)
Autumn_hour_PVoutput = np.array(Autumn_PVoutput)
Autumn_hour_Load = np.array(Autumn_Load)
Autumn_hour_Price = np.array(Autumn_Price)

#reshape成24*91 --> 24小時，91天
Autumn_hour_Charge = Autumn_hour_Charge.reshape(24, 91)
Autumn_hour_Discharge = Autumn_hour_Discharge.reshape(24, 91)
Autumn_hour_Purchase = Autumn_hour_Purchase.reshape(24, 91)
Autumn_hour_PVoutput = Autumn_hour_PVoutput.reshape(24, 91)
Autumn_hour_Load = Autumn_hour_Load.reshape(24, 91)
Autumn_hour_Price = Autumn_hour_Price.reshape(24, 91)

Autumn_average_Charge, Autumn_average_Discharge, Autumn_average_Purchase, Autumn_average_PVoutput, Autumn_average_Load, Autumn_average_Price = [], [], [], [], [], []

for hour in range(0, 24):
    Autumn_average_Charge.append(round(np.average(Autumn_hour_Charge[hour]), 4))
    Autumn_average_Discharge.append(round(np.average(Autumn_hour_Discharge[hour]), 4))
    Autumn_average_Purchase.append(round(np.average(Autumn_hour_Purchase[hour]), 4))
    Autumn_average_PVoutput.append(round(np.average(Autumn_hour_PVoutput[hour]), 4))
    Autumn_average_Load.append(round(np.average(Autumn_hour_Load[hour]), 4))
    Autumn_average_Price.append(round(np.average(Autumn_hour_Price[hour]), 4))
    

width1 = 0.25  #條狀圖單一條的寬度
plt.figure(figsize = (10, 10))  #設定圖表長寬
plt.subplot(2,1,1)  #每日圖表以兩張圖表組合在一起，此為第一張
list1 = [x - width1/2 for x in range(len(list_hour))]  #因兩項資料放在一起，要共用資料寬度，所以一人一半
list2 = [x + width1/2 for x in range(len(list_hour))]
#.iloc[index_count:index_count+24] => 取得列索引值的對應資料
#["Purchase"] => 取得對應column name值的資料
#bar(橫軸值, 縱軸值, 條寬度, 條顏色,資料標籤)
plt.bar(list1, Autumn_average_Purchase, width1, color = "cyan", label = "Purchase")  #購電量資料
plt.bar(list2, Autumn_average_Load, width1, color = "red", label = "Load")  #負載量資料
plt.xticks(range(len(list_hour)), labels = list_hour)  #x軸座標值
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.title("Autumn Average")  #圖表標題
plt.xlabel("Hours")  #x軸座標標籤
plt.ylabel("kW/h")  #y軸座標標籤
plt.legend(loc = "upper left")  #資料標籤顯示位置
plt.grid()  #圖片結果以網格顯示
ax2 = plt.twinx()  #定義第二組不同呈現方式的資料
ax2.set_ylabel("US $/kWh")  #第二組y軸座標標籤
#plt(橫軸值, 縱軸值, 折線顏色與型態, 資料標籤)
plt.plot(list_hour, Autumn_average_Price, "g-.*", label = "Price")  #購電價資料
plt.ylim(0, 0.2)
plt.legend(loc = "upper right")

width2 = 0.25
plt.subplot(2,1,2)
list1 = [x - width2/2 for x in range(len(list_hour))]  #因兩項資料放在一起，要共用資料寬度，所以一人一半
list2 = [x + width2/2 for x in range(len(list_hour))]
plt.bar(list1, Autumn_average_Charge, width2, color = "orange", label = "Charge")  #充電量資料
plt.bar(list2, Autumn_average_Discharge, width2, color = "blue", label = "Discharge")  #放電量資料
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.xlabel("Hours")
plt.ylabel("kW/h")
plt.legend(loc = "upper left")
plt.grid()
ax2 = plt.twinx()
ax2.set_ylabel("kW/h")
plt.plot(list_hour, Autumn_average_PVoutput, "r-.*", label = "PV output")  #光伏輸出量資料
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.legend(loc = "upper right")

plt.savefig("result_figures/Autumn.jpg")
plt.close()  #將製作完的圖表關閉，避免佔據記憶體空間

In [ ]:
#冬天12,1,2月
Winter_Charge, Winter_Discharge, Winter_Purchase, Winter_PVoutput, Winter_Load, Winter_Price = [], [], [], [], [], []

for hour in range(0, 24):
    month = 11
    for index in range(month_begin_index[month]+hour, (month_begin_index[month]+(N_m[month]*24)), 24):
        #print(hour, month, index)
        Winter_Charge.append(Result.iloc[index, :]['Charge'])
        Winter_Discharge.append(Result.iloc[index, :]['Discharge'])
        Winter_Purchase.append(Result.iloc[index, :]['Purchase'])
        Winter_PVoutput.append(Result.iloc[index, :]['PVoutput'])
        Winter_Load.append(Result.iloc[index, :]['Load'])
        Winter_Price.append(Result.iloc[index, :]['Price'])
    for month in range(0, 2):
        for index in range(month_begin_index[month]+hour, (month_begin_index[month]+(N_m[month]*24)), 24):
            #print(hour, month, index)
            Winter_Charge.append(Result.iloc[index, :]['Charge'])
            Winter_Discharge.append(Result.iloc[index, :]['Discharge'])
            Winter_Purchase.append(Result.iloc[index, :]['Purchase'])
            Winter_PVoutput.append(Result.iloc[index, :]['PVoutput'])
            Winter_Load.append(Result.iloc[index, :]['Load'])
            Winter_Price.append(Result.iloc[index, :]['Price'])

Winter_hour_Charge = np.array(Winter_Charge)
Winter_hour_Discharge = np.array(Winter_Discharge)
Winter_hour_Purchase = np.array(Winter_Purchase)
Winter_hour_PVoutput = np.array(Winter_PVoutput)
Winter_hour_Load = np.array(Winter_Load)
Winter_hour_Price = np.array(Winter_Price)

Winter_hour_Charge = Winter_hour_Charge.reshape(24, 91)
Winter_hour_Discharge = Winter_hour_Discharge.reshape(24, 91)
Winter_hour_Purchase = Winter_hour_Purchase.reshape(24, 91)
Winter_hour_PVoutput = Winter_hour_PVoutput.reshape(24, 91)
Winter_hour_Load = Winter_hour_Load.reshape(24, 91)
Winter_hour_Price = Winter_hour_Price.reshape(24, 91)

Winter_average_Charge, Winter_average_Discharge, Winter_average_Purchase, Winter_average_PVoutput, Winter_average_Load, Winter_average_Price = [], [], [], [], [], []

for hour in range(0, 24):
    Winter_average_Charge.append(round(np.average(Winter_hour_Charge[hour]), 4))
    Winter_average_Discharge.append(round(np.average(Winter_hour_Discharge[hour]), 4))
    Winter_average_Purchase.append(round(np.average(Winter_hour_Purchase[hour]), 4))
    Winter_average_PVoutput.append(round(np.average(Winter_hour_PVoutput[hour]), 4))
    Winter_average_Load.append(round(np.average(Winter_hour_Load[hour]), 4))
    Winter_average_Price.append(round(np.average(Winter_hour_Price[hour]), 4))
    
    
width1 = 0.25  #條狀圖單一條的寬度
plt.figure(figsize = (10, 10))  #設定圖表長寬
plt.subplot(2,1,1)  #每日圖表以兩張圖表組合在一起，此為第一張
list1 = [x - width1/2 for x in range(len(list_hour))]  #因兩項資料放在一起，要共用資料寬度，所以一人一半
list2 = [x + width1/2 for x in range(len(list_hour))]
#.iloc[index_count:index_count+24] => 取得列索引值的對應資料
#["Purchase"] => 取得對應column name值的資料
#bar(橫軸值, 縱軸值, 條寬度, 條顏色,資料標籤)
plt.bar(list1, Winter_average_Purchase, width1, color = "cyan", label = "Purchase")  #購電量資料
plt.bar(list2, Winter_average_Load, width1, color = "red", label = "Load")  #負載量資料
plt.xticks(range(len(list_hour)), labels = list_hour)  #x軸座標值
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.title("Winter Average")  #圖表標題
plt.xlabel("Hours")  #x軸座標標籤
plt.ylabel("kW/h")  #y軸座標標籤
plt.legend(loc = "upper left")  #資料標籤顯示位置
plt.grid()  #圖片結果以網格顯示
ax2 = plt.twinx()  #定義第二組不同呈現方式的資料
ax2.set_ylabel("US $/kWh")  #第二組y軸座標標籤
#plt(橫軸值, 縱軸值, 折線顏色與型態, 資料標籤)
plt.plot(list_hour, Winter_average_Price, "g-.*", label = "Price")  #購電價資料
plt.ylim(0, 0.2)
plt.legend(loc = "upper right")

width2 = 0.25
plt.subplot(2,1,2)
list1 = [x - width2/2 for x in range(len(list_hour))]  #因兩項資料放在一起，要共用資料寬度，所以一人一半
list2 = [x + width2/2 for x in range(len(list_hour))]
plt.bar(list1, Winter_average_Charge, width2, color = "orange", label = "Charge")  #充電量資料
plt.bar(list2, Winter_average_Discharge, width2, color = "blue", label = "Discharge")  #放電量資料
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.xlabel("Hours")
plt.ylabel("kW/h")
plt.legend(loc = "upper left")
plt.grid()
ax2 = plt.twinx()
ax2.set_ylabel("kW/h")
plt.plot(list_hour, Winter_average_PVoutput, "r-.*", label = "PV output")  #光伏輸出量資料
y = [0, 2000, 4000, 6000, 8000, 10000]
y_tick_labels = ['0', '2,000', '4,000', '6,000', '8,000', '10,000']
plt.yticks(y, y_tick_labels)  #y軸座標數值範圍
plt.legend(loc = "upper right")

plt.savefig("result_figures/Winter.jpg")
plt.close()  #將製作完的圖表關閉，避免佔據記憶體空間